In [1]:
##########################################################################
## The Connection Part

class GameConnection():
    def get_message(self, timeout=None):
        return None
    def send_message(self, msg):
        pass
    def close(self):
        pass

import socket
from scapy.all import *
class TS():
    def __init__(self, pkt):
        self.init = True
        if 'Raw' in pkt:
#             self.data = pkt[Raw].load
            self.data = b''
            self.nseq = pkt[TCP].seq + len(self.data)
        else:
            self.data = b''
            if pkt[TCP].flags & 2 == 2:
                self.nseq = pkt[TCP].seq  + 1
    def add_packet(self, pkt):
        ret = []
        if pkt[TCP].seq != self.nseq:
#             print(pkt[TCP].seq, self.nseq)
#             print('Oops')
            return ret
        if 'Raw' in pkt:
            self.data += pkt[Raw].load
            self.nseq = pkt[TCP].seq + len(pkt[Raw].load)
            if self.init:
                head = self.data.find(b'\r\n\r\n')
                if head > 0:
                    begin = head+4
                    self.init = False
                    self.c = begin
            if not self.init:
                for i in range(5):
                    found_frame = False
                    if len(self.data) - self.c >= 2:
                        ws_op, ws_len = struct.unpack('BB', self.data[self.c:self.c+2])
                        ll = 0
                        mm = 0
                        mask_flag = ws_len & 0x80
                        ws_len = ws_len & 0x7f
                        if mask_flag > 0: mm = 4
                        if ws_len == 126:
                            ll = 2
                            if len(self.data) - self.c >= 2+ll+4:
                                ws_len = struct.unpack('>H', self.data[self.c+2:self.c+4])[0]
                        if len(self.data) - self.c >= 2+ll+mm+ws_len:
                            payload = self.data[self.c+2+ll+mm:self.c+2+ll+mm+ws_len]
                            if mask_flag == 0:
                                masks = b'\x00\x00\x00\x00'
                            else:
                                masks = self.data[self.c+2+ll:self.c+2+ll+4]
                            if mask_flag: 
                                payload = unmask_payload(payload, masks)
                            if ws_op == 0x82:
                                ret.append((payload, mask_flag))
                            found_frame = True
                            self.c += 2+ll+mm+ws_len
                            self.data = self.data[self.c:]
                            self.c = 0
                    if not found_frame:
                        break
        else:
            if pkt[TCP].flags & 3 : # SYN or FIN
                self.nseq = pkt[TCP].seq  + 1
        return ret

class PcapGameConnection(GameConnection):
    def __init__(self, filename=None, live=False):
        self.msgs = []
        self.to = None
        if filename:
            self.pkts = []
            def dp(pkt):
                self.pkts.append(pkt)
            sniff(prn=dp, offline=filename)
#             print(self.pkts)
            self.pkts = iter(self.pkts)
        if live:
            self.sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.sk.bind(('0.0.0.0', 8989))
        def message_gen():
            ss = {}
#             n = 0
            while True:
#                 print('n -> ', n)
                if self.msgs:
                    yield self.msgs[0]
                    del self.msgs[0]
                else:
                    if filename and not live:
                        p = next(self.pkts)
                    if live:
                        try:
                            self.sk.settimeout(self.to)
                            data, addr = self.sk.recvfrom(8192)
                        except socket.timeout:
                            yield None, 0
                        except:
                            self.sk = None
                        p = Ether(data)
                    if (p[TCP].sport, p[TCP].dport) not in ss:
                        s = TS(p)
                        ss[(p[TCP].sport, p[TCP].dport)] = s
                    else:
                        s = ss[(p[TCP].sport, p[TCP].dport)]
                        ms = s.add_packet(p)
                        self.msgs += ms
#                 n += 1
        self.msg_gen = message_gen()
    def deal_pkt(self):
        pass
    def get_message(self, timeout=None):
        self.to = timeout
        while True:
            msg, mask = next(self.msg_gen)
            if msg == None:    ## when live mode and no notify
                return None
            if not timeout:
                if mask == 0:
                    return msg
                else:
                    if type(PKT.decode(msg)) == HEAT_BEAT: continue
                    if msg != self.send_buf:
                        cprint('################### send payload mismatch ######################', 'yellow')
                        print(msg)
                        print(self.send_buf)
                        print(PKT.decode(msg))
                        print(PKT.decode(self.send_buf))
                        print('----------------------------------------------------------------')
                    else:
                        print('################### send payload matched ######################')
                        print(PKT.decode(self.send_buf))
                        print('***************************************************************')
            else:
                if type(PKT.decode(msg)) == SUB_S_OPERATE_NOTIFY:
                    return msg
                else:
                    self.msgs.insert(0, msg)
                    return None
                    
    def send_message(self, buf):
        if type(PKT.decode(buf)) == HEAT_BEAT: return
        self.send_buf = buf

from websocket import create_connection,WebSocketTimeoutException
class WSGameConnection(GameConnection):
    def __init__(self):
        self.ws = create_connection("ws://73.hanyou.com:17301/")
    def send_message(self, buf):
        self.ws.send(buf)
    def get_message(self, timeout=None):
        try:
            self.ws.settimeout(timeout)
            return self.ws.recv()
        except WebSocketTimeoutException:
            return None
    def close(self):
        self.ws.close()

In [2]:
##########################################################################
## The AI Part

weights = [10, 20, 30, 40, 50, 40, 30, 20, 10, 
          10, 20, 30, 40, 50, 40, 30, 20, 10, 
          10, 20, 30, 40, 50, 40, 30, 20, 10, 
          5, 5, 5, 5, 5, 5, 5, 5, 5]

def discard_tile(hand, dead):
    rs = calculate_outs(hand, dead)
    rs.sort(key=lambda x:(-x['tiles_count'], weights[x['discard']]))
    return rs[0]
#     return rs[0]['discard']

def give_me_tile(candi, hand, dead):
    ret = []
    for i in candi:
        if should_op_tile(hand, dead, i):
            ret.append(i)
    return ret

def should_op_tile(hand_orig, dead_orig, tile):
    hand = list(hand_orig)
    hand[33] += 1
    rs = calculate_outs(hand, dead_orig)
#     rs.sort(key=lambda x:(-x['tiles_count'], weights[x['discard']]))
    best_opt = None
    for opt in rs:
        if opt['discard'] == 33:
            best_opt = opt
    if not best_opt: return None
    ret = None
#     print(best_opt)
    if hand[tile] >1 :
        hand = list(hand_orig)
        dead = list(dead_orig)
        hand[tile] -= 2
        dead[tile] += 3
        rs = calculate_outs(hand, dead)
        opt = rs[0]
#         print(opt)
        if opt['shanten'] < best_opt['shanten'] or opt['tiles_count'] > best_opt['tiles_count']:
            best_opt = opt
            ret = (tile, tile)
    if hand[tile+1] > 0 and hand[tile+2] > 0 and tile < 27 and (tile)//9 == (tile+2)//9:
        hand = list(hand_orig)
        dead = list(dead_orig)
        hand[tile+1] -= 1
        hand[tile+2] -= 1
        dead[tile] += 1
        dead[tile+1] += 1
        dead[tile+2] += 1
        rs = calculate_outs(hand, dead)
        opt = rs[0]
#         print(opt)
        if opt['shanten'] < best_opt['shanten'] or opt['tiles_count'] > best_opt['tiles_count']:
            best_opt = opt
            ret = (tile+1, tile+2)
    if hand[tile+1] > 0 and hand[tile-1] > 0 and tile < 27 and (tile+1)//9 == (tile-1)//9:
        hand = list(hand_orig)
        dead = list(dead_orig)
        hand[tile+1] -= 1
        hand[tile-1] -= 1
        dead[tile] += 1
        dead[tile+1] += 1
        dead[tile-1] += 1
        rs = calculate_outs(hand, dead)
        opt = rs[0]
#         print(opt)
        if opt['shanten'] < best_opt['shanten'] or opt['tiles_count'] > best_opt['tiles_count']:
            best_opt = opt
            ret = (tile-1, tile+1)
    if hand[tile-1] > 0 and hand[tile-2] > 0 and tile < 27 and (tile)//9 == (tile-2)//9:
        hand = list(hand_orig)
        dead = list(dead_orig)
        hand[tile-1] -= 1
        hand[tile-2] -= 1
        dead[tile] += 1
        dead[tile-1] += 1
        dead[tile-2] += 1
        rs = calculate_outs(hand, dead)
        opt = rs[0]
#         print(opt)
        if opt['shanten'] < best_opt['shanten'] or opt['tiles_count'] > best_opt['tiles_count']:
            best_opt = opt
            ret = (tile-2, tile-1)
    return ret
    

def calculate_outs(hand, dead):
    """
    hand: 34 array
    dead: 34 array
    """
    if sum(hand) == 1:
        t = 0
        for i in range(34):
            if hand[i]:
                t = i; break
        return [{'shanten':0, 'waiting':[1], 'tiles_count':99, 'discard':t}]
    meld_num = 4 - sum(hand)//3
    results = []
#     print(hand, dead)
    hand_bak = list(hand)
    s_in = shanten(hand, meld_num)
#     print('--> s_in ', s_in)
    should_run_2 = False
    for k in (1, 2):
        for candi in range(34):
            if not hand[candi]:
                continue
            hand[candi] -= 1
            hand[33] += 1
            s = shanten(hand, meld_num)
            hand[33] -= 1
            if s > s_in and not should_run_2:
                hand[candi] += 1
                continue

            waiting = []
            tiles_count = 0
            for j in range(0, 34):
                if candi == j or hand[j] == 4:
                    continue
                hand[j] += 1
                if shanten(hand, meld_num) < s:
                    waiting.append(j)
    #                 print((4 - hand_bak[j] - dead[j]), TilesConverter.to_one_line_string([j*4]), hand_bak[j], dead[j], j)
                    tiles_count += (4 - hand_bak[j] - dead[j])
                hand[j] -= 1
            hand[candi] += 1

            if waiting:
                discard_opt = {'shanten':s, 'waiting':waiting, 'tiles_count':tiles_count, 'discard':candi}
                results.append(discard_opt)
        results.sort(key=lambda x:(x['shanten'], -x['tiles_count'], weights[x['discard']]))
        if results:
            if should_run_2:
                s = set(results[0]['waiting'])
                for i in results:
                    s = s.intersection(i['waiting'])
                if s:
                    t = s.pop()
                    if hand[t]: 
#                         for i in results:
#                             print(i)
                        return [{'shanten':1, 'waiting':[], 'tiles_count':99, 'discard':t}]
            return results
        else:
            should_run_2 = True
    if not results: ## in case =======> my hand:  9条|9条|9条|9条|1万|2万|3万|混|
        for i in range(34):
            if hand[i] == 4:
                return [{'shanten':1, 'waiting':[], 'tiles_count':99, 'discard':i}]
    return results

# shanten = shanten_list

In [101]:
from mahjong.tests_mixin import TestMixin
from mahjong.utils import find_isolated_tile_indices
from mahjong.tile import TilesConverter
import mahjong.utils

a = TestMixin()
# tiles = a._string_to_136_array(sou='11134567', pin='159', man='45')
tiles_34 = a._string_to_34_array(sou='', man='567', pin='234444', honors='')

t = TilesConverter.string_to_136_array(man='4')[0]//4
print(t)
print(tiles_34)
should_op_tile(tiles_34, [0]*34, t)
# shanten(tiles_34, 1)
# calculate_outs(tiles_34, [0]*34)
# # # discard_tile(tiles_34, [0]*34)



# d = [0] * 34

# h = tiles_34

# should_op_tile(h, d, t)

# do = discard_tile(tiles_34, [0]*34)
# # give_me_tile(do['waiting'], tiles_34, d)
# do

3
[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
##########################################################################
## The Game Logic Part

import json, http.client, struct, os
from codes import *

def login_3(mac):
    conn = http.client.HTTPSConnection("web.hanyou.com")
    conn.request("GET", "/m/login_t.do?mac="+mac+"&state=%E4%B8%8A%E6%B5%B7&city=%E4%B8%8A%E6%B5%B7&lineid=100000")
    content = conn.getresponse().read()
    content = json.loads(content.decode('utf-8'))
    return content

def login(mac):
    conn = http.client.HTTPSConnection("web.hanyou.com")
    conn.request("GET", "/m/login_t.do?mac="+mac+"&state=%E4%B8%8A%E6%B5%B7&city=%E4%B8%8A%E6%B5%B7&lineid=100000")
    content = conn.getresponse().read()
    content = json.loads(content.decode('utf-8'))
    headers = {"Cookie":"JSESSIONID=%s"%content['sessionid'],"session_id":"%s"%content['sessionid'],
              "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.1.1; SM-N9005 Build/NJH47F)"}
    if content['score'] < 500:
        conn.request("POST", "/reloadGameMoney.do", '', headers)
        sync_ret = conn.getresponse().read()
        sync_ret = json.loads(sync_ret.decode('utf-8'))
        cprint(sync_ret, 'yellow')
    conn = http.client.HTTPConnection("web.hanyou.com")
    conn.request("POST", "/login/synconline.do?gid=73&sid=17301", '', headers)
    sync_ret = conn.getresponse().read()
    sync_ret = json.loads(sync_ret.decode('utf-8'))
    return content

def prize(mac, count=100, take_all=False):
    para_map = {100:295, 500:235, 1000:91, 5000:445, 10000:92}
    conn = http.client.HTTPSConnection("web.hanyou.com")
    conn.request("GET", "/m/login_t.do?mac="+mac+"&state=%E4%B8%8A%E6%B5%B7&city=%E4%B8%8A%E6%B5%B7&lineid=100000")
    content = conn.getresponse().read()
    content = json.loads(content.decode('utf-8'))
    medals = content['medals']
    if take_all:
        count = medals
    ids = []
    keys = list(para_map.keys())
    keys.sort(reverse=True)
    for i in keys:
        while count >= i:
            ids.append(para_map[i])
            count -= i
    headers = {"Cookie":"JSESSIONID=%s"%content['sessionid'],"session_id":"%s"%content['sessionid'],
              "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.1.1; SM-N9005 Build/NJH47F)"}
    for i in ids:
        conn.request("POST", "/shop/exchange.do?orders.type=1&orders.prizeId=%d&orders.mobile=&orders.exchangeMode=0&orders.address=&orders.userName=&orders.preAddress=&orders.jdProvinces=0&orders.jdCity=0&orders.jdCounty=0&orders.jdTown=0"%i, '', headers)
        sync_ret = conn.getresponse().read()
        sync_ret = json.loads(sync_ret.decode('utf-8'))
    print(sync_ret)

# login('ffffffff-b319-4c6f-ffff-ffffa7b1fa34')

from termcolor import colored, cprint
class VGame():
    def __init__(self, isus=False, me_richer=False, me_id=None, enemy_id=None):
        self.hand = []
        self.me_out_list = []
        self.enemy_out_list = []
        self.double = 0
        self.isus = isus
        self.me_richer = me_richer
        self.enemy_id = enemy_id
        self.me_id = me_id
        self.can_hu = False
    def me_out(self, tile):
        if tile >= 0x35 and tile <= 0x37: self.double += 1
        self.me_out_list.append(tile)
        self.hand.remove(tile)
        print('==========> We out ', CARDS[tile])
        self.print_myhand()
    def me_worst_tile(self):
        t = None
        aux = ''
        if self.me_richer:
            l = enemy_wanted_list(self.enemy_id)
            if not l:
                l = [99]
            for i in self.hand:
                if i >= 0x35 and i <=0x37:  # double
                    t =i; break
            if not t: ## no double found
                if l[-1] ==0:   ## enemy can win but don't want to, for there is too few double
                    for i in self.hand:
                        if i not in l:
                            t = i; break
                else:
                    for i in self.hand:
                        if i in l:
                            t = i; break
            if not t:
                for i in self.hand:
                    if i < 0x35:
                        t = i; break
        else:
            ####
            aux = ''
            if 0x35 in self.hand:
                t = 0x35
            if 0x36 in self.hand:
                t = 0x36
            if 0x37 in self.hand:
                t = 0x37
            if not t:
                self.print_myhand()
                print(list_to_34(self.hand))
                h = list_to_34(self.hand)
                d = list_to_34(self.me_out_list+self.enemy_out_list)
                do = discard_tile(h, d)
                wants = give_me_tile(do['waiting'], h, d)
                if self.isus:
                    rw = [tile_from_34[i] for i in wants]
                    if self.can_hu and not self.should_hu():
                        rw.append(0)
                    redis.set(self.me_id, rw)
                t = tile_from_34[do['discard']]
                w = '|'.join([CARDS[tile_from_34[i]] for i in do['waiting']])
                w2 = '|'.join([CARDS[tile_from_34[i]] for i in wants])
                aux = '    shanten[%d]  waiting[%s]  desperate[%s]' % (do['shanten'], w, w2)
        cprint('##############=============> We suggest %s times[%d] %s' % (CARDS[t], 2**self.double, aux), 'green')
        return t
    def enemy_out(self, tile):
        if tile >= 0x35 and tile <= 0x37: self.double += 1
        self.enemy_out_list.append(tile)
        print('==========> They out ', CARDS[tile])
    def me_init_hand(self, tiles):
        self.hand = tiles
    def me_draw_tile(self, tile):
        self.hand.append(tile)
        print('==========> We draw ', CARDS[tile])
    def operate_tile(self, tile, mask):
        if self.me_richer:
            return 0
        opt = should_op_tile(list_to_34(self.hand), list_to_34(self.me_out_list+self.enemy_out_list), tile_to_34[tile])
        if opt:
            t1, t2 = [tile_from_34[i] for i in opt]
            rs = [t1, t2, tile]
            rs.sort()
            rs = [CARDS[i] for i in rs]
            cprint('##############=============> We suggest '+ ''.join(rs), 'green')
            d = tile - t1
            if d == 0:
                return WIK_PENG
            elif d == -1:
                return WIK_LEFT
            elif d == 1:
                return WIK_CENTER
            elif d == 2:
                return WIK_RIGHT
        else:
            cprint('##############=============> We suggest 过', 'green')
            return 0
    def print_myhand(self):
        self.hand.sort()
        ret = ''
        for i in self.hand:
            ret += CARDS[i]+'|'
        print('=======> my hand: ', ret)
    def no_op(self):
        cprint('##############=============> We suggest 过', 'green')
    def chi_hu(self):
        self.can_hu = True
        cprint('##############=============> We suggest 胡', 'green')
    def zi_mo(self):
        cprint('##############=============> We suggest 自摸', 'green')
    def fan(self):
        return self.double
    def should_hu(self):
        if self.me_richer:
            return False
        if not self.isus:
            return True
        if self.fan() < 7:
            return False
        return True

min_workers = 2
def play_game(mac=None, filename=None, live=False):
    if mac:
        conn = WSGameConnection()
        c = login(mac) ## login('ffffffff-e184-87b9-ffff-ffffa7b1fa34')
        userID, pwd, my_score = int(c['userId']), c['pwd'], c['score']
    else:
        conn = PcapGameConnection(filename=filename, live=live)
        userID, pwd, my_score = 123, 'password', 999
    login_pkt = SUB_GS_LOGON_USERID(userID, pwd)
    conn.send_message(login_pkt.ser())
    me_id = 0
    while True:
        msg = conn.get_message()
        print('==> ', msg)
        msg = PKT.decode(msg)
        print(msg)
        if type(msg) == SUB_GS_USER_SCORE:
            me_id = msg.dwUserID
            my_score = msg.lScore
            print('login done')
            break
#     return
    def join_table():
        go_together(userID, min_workers)
        print('gogogo')
        conn.send_message(SUB_GS_USER_JOIN_RANDOM_LIST_SYN().ser())
    join_table()
    me_chair = -1
    g = None
    def out_card():
        t = g.me_worst_tile()
        conn.send_message(SUB_C_OUT_CARD(t).ser())
    while True:
        msg = conn.get_message()
        msg = PKT.decode(msg)
        print(msg)
        if type(msg) == SUB_S_START_SEND_CARD:
            g = VGame(is_usus, me_richer, me_id, enemy_userID)
            g.me_init_hand(msg.tiles)
            if msg.wBankerUser == me_chair:
                out_card()
            else:
                g.hand.remove(0)
        elif type(msg) == HEAT_BEAT:
            conn.send_message(msg.ser())  # send it back
        elif type(msg) == SUB_S_OUT_CARD:
            if me_chair == msg.user:
                g.me_out(msg.tile)
            else:
                g.enemy_out(msg.tile)
        elif type(msg) == SUB_GS_USER_COME:
            enemy_userID, enemy_score = msg.info['id'], msg.info['lScore']
            if is_us(enemy_userID):
                is_usus = True
            else:
                is_usus = False
            me_richer = is_usus and my_score > enemy_score
        elif type(msg) == SUB_GS_USER_STATUS:
            if me_id == msg.userID:
                me_chair = msg.chairID
                if msg.status == 1:
                    if os.path.exists('stop'): break
                    join_table()
                if msg.status == 2:
                    conn.send_message(SUB_GS_INFO().ser())
        elif type(msg) == SUB_S_SEND_CARD:
            if me_chair == msg.user:
                g.me_draw_tile(msg.tile)
                t = g.me_worst_tile(); t2 = t
                msg = conn.get_message(timeout=0.03)
                need_out_card = True
                if msg: 
                    msg = PKT.decode(msg)
                    if type(msg) == SUB_S_OPERATE_NOTIFY:
                        print(msg)
                        m = msg.cbActionMask
                        t = msg.cbActionCard
                        op = SUB_C_OPERATE_CARD()
                        op.tile = t
                        if m & WIK_CHI_HU:
                            g.chi_hu()
                            op.action = WIK_CHI_HU
                            need_out_card = False
                        elif m & WIK_ZI_MO:
                            g.zi_mo()
                            op.action = WIK_ZI_MO
                            need_out_card = False
                        elif m & WIK_GANG:
                            g.no_op()
                            op.action = 0
                            op.tile = 0
                        if m & WIK_CHI_HU and not g.should_hu():
                            g.no_op()
                            op.action = 0
                            op.tile = 0
                            t = t2
                        conn.send_message(op.ser())
                if need_out_card: conn.send_message(SUB_C_OUT_CARD(t).ser())
        elif type(msg) == SUB_S_OPERATE_NOTIFY:
            m = msg.cbActionMask
            t = msg.cbActionCard
            op = SUB_C_OPERATE_CARD()
            op.tile = t
            actions = WIK_LEFT | WIK_CENTER | WIK_RIGHT | WIK_PENG
            if m & WIK_CHI_HU:
                g.chi_hu()
                op.action = WIK_CHI_HU
                if not g.should_hu():
                    if actions & m:
                        op.action = g.operate_tile(t, m)
                        if op.action == 0: op.tile = 0
            elif m & WIK_ZI_MO:
                g.zi_mo()
                op.action = WIK_ZI_MO
            elif actions & m:
                op.action = g.operate_tile(t, m)
                if op.action == 0: op.tile = 0
            elif m & WIK_GANG:
                g.no_op()
                op.action = 0
                op.tile = 0
            conn.send_message(op.ser())
        elif type(msg) == SUB_S_OPERATE_RESULT:
            cbOperateCode = msg.cbOperateCode
            cbOperateCard = msg.cbOperateCard
            if cbOperateCode:
                cards = []
                if cbOperateCode == WIK_LEFT:
                    cards = [cbOperateCard+1, cbOperateCard+2]
                elif cbOperateCode == WIK_CENTER:
                    cards = [cbOperateCard-1, cbOperateCard+1] 
                elif cbOperateCode == WIK_RIGHT:
                    cards = [cbOperateCard-2, cbOperateCard-1] 
                elif cbOperateCode == WIK_PENG:
                    cards = [cbOperateCard, cbOperateCard] 
                elif cbOperateCode == WIK_GANG:
                    cards = [cbOperateCard, cbOperateCard, cbOperateCard]
                if msg.wOperateUser == me_chair:
                    for i in cards:
                        g.me_out(i)
                    out_card()
            else:
                if len(g.hand) % 3 == 2:  # when we can zi_mo but we choose not to
                    out_card()
        elif type(msg) == SUB_S_GAME_END:
            txt = 'GameScore[%d] WinTimes[%d]'%(msg.lGameScore[me_chair], msg.lWinTimes[me_chair])
            cprint(txt, 'green')
        elif type(msg) == SUB_GS_USER_SCORE and msg.dwUserID == me_id:
            txt = 'MyScore[%d] WinLostRate[%d/%d]'%(msg.lScore, msg.lWinCount, msg.lLostCount)
            my_score = msg.lScore
            cprint(txt, 'green')
    conn.close()

In [29]:
####################################################################
## redis part

from redis import StrictRedis
import time
import random

redis = StrictRedis(host='localhost', port=6379)

min_worker = 3

ps = redis.pubsub()
ps.subscribe('go')


def go_together(worker_name, min_worker=3):
    while True:                ## consume old messages
        m = ps.get_message()
        if not m:
            break
    redis.hset('worker_state', worker_name, 'ready')
    ws = redis.hgetall('worker_state')
    ready_workers = [i for i in ws if ws[i] == b'ready']
    if len(ready_workers) >= min_worker:
#         print('gogogogooggoogogogogog')
        redis.publish('go', 'gogogo')
    else:
        for m in ps.listen():
            if m['type'] == 'message':
                print(m)
#                 ps.unsubscribe('go')
                break
    redis.hset('worker_state', worker_name, 'busy')
    
def is_us(userid):
    return redis.hget('worker_state', userid)
    
def enemy_wanted_list(userid):
    ret = redis.get(userid)
    if ret:
        ret = json.loads(redis.get(userid).decode('utf-8'))
    return ret

In [253]:
is_us()

Done


In [4]:
prize('ffffffff-e184-87b1-ffff-ffffa7b1fa34',500, take_all=True)

{'orderId': 36012284, 'msg': 'success', 'code': 0}


In [16]:
def login_3(mac, port=17301):
    conn = http.client.HTTPSConnection("web.hanyou.com")
#     conn.request("GET", "/m/login_t.do?mac="+mac+"&state=%E4%B8%8A%E6%B5%B7&city=%E4%B8%8A%E6%B5%B7&lineid=100000")
    conn.request("GET", "/m/login.do?phone=13917909443&mac=00000000-1510-f4c0-ffff-ffffb09c2237&userId=56762903&type=0")
    content = conn.getresponse().read()
    content = json.loads(content.decode('utf-8'))
    headers = {"Cookie":"JSESSIONID=%s"%content['sessionid'],"session_id":"%s"%content['sessionid'],
              "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.1.1; SM-N9005 Build/NJH47F)"}
    if content['score'] < 500:
        conn.request("POST", "/reloadGameMoney.do", '', headers)
        sync_ret = conn.getresponse().read()
        sync_ret = json.loads(sync_ret.decode('utf-8'))
        cprint(sync_ret, 'yellow')
    conn = http.client.HTTPConnection("web.hanyou.com")
    print('$$$$$$$$--> ', port)
    conn.request("POST", "/login/synconline.do?gid=73&sid=%s"%port, '', headers)
    sync_ret = conn.getresponse().read()
    sync_ret = json.loads(sync_ret.decode('utf-8'))
    return content

In [7]:
login_3('ffffffff-88a0-47cf-ffff-ffff8742eadc')

{'account': '84073e262a98b90e76549ab153bf594f',
 'chgNick': 1,
 'code': 0,
 'faceUrl': 'http://static.hanyou.com/face/default/girl1.png?v=1',
 'lastDepositScore': 0,
 'logoutDeposit': 0,
 'matchKey': 'ddb07ef37a293988f628faed6ddebdad',
 'medals': 0,
 'money': 0,
 'newAcc': True,
 'nickName': '游客60306481',
 'openBank': 0,
 'phone': '',
 'pwd': '414f1c2c7d4ec09a982876b40b2bed9e',
 'score': 0,
 'sessionid': '62D7D57639153692FCB2CDD3B6BFAE52',
 'titleLevel': 0,
 'userId': 60306481,
 'vip': 0}

In [10]:
ccc = '{"medals":70503,"code":0,"openBank":1,"nickName":"游客56762903","lastDepositScore":0,"matchKey":"5259d9e8aaa845da7ec4de9401ed6979","sessionid":"C15A1E1166B1816118851F9F28DF68B7","logoutDeposit":0,"userId":56762903,"faceUrl":"http://static.hanyou.com/face/default/girl14.png?v=14","score":237536,"chgNick":1,"money":0,"phone":"13917909443","titleLevel":1,"pwd":"ff15efd9e54ea8dde712044e1e127fea","vip":0,"account":"2140eb3326889764b61e5af82d89d0a5","bankScore":0}'
json.loads(ccc)

{'account': '2140eb3326889764b61e5af82d89d0a5',
 'bankScore': 0,
 'chgNick': 1,
 'code': 0,
 'faceUrl': 'http://static.hanyou.com/face/default/girl14.png?v=14',
 'lastDepositScore': 0,
 'logoutDeposit': 0,
 'matchKey': '5259d9e8aaa845da7ec4de9401ed6979',
 'medals': 70503,
 'money': 0,
 'nickName': '游客56762903',
 'openBank': 1,
 'phone': '13917909443',
 'pwd': 'ff15efd9e54ea8dde712044e1e127fea',
 'score': 237536,
 'sessionid': 'C15A1E1166B1816118851F9F28DF68B7',
 'titleLevel': 1,
 'userId': 56762903,
 'vip': 0}

In [44]:
# e = b'2\x00\x00E\x100\x120_\xcf_\x03\x00034225f4edd7a6ce9814e8b63406efa2\x00\x03\x00\x00\x00'
# p = encode_MAIN_GS_LOGON__SUB_GS_LOGON_USERID(56610655, '034225f4edd7a6ce9814e8b63406efa2')
# m = encode_message(p, codes_c2n['MAIN_GS_LOGON'], codes_c2n['SUB_GS_LOGON_USERID'])
# assert m == e

play_game(filename = '/tmp/0605.pcap')
# play_game(mac = 'ffffffff-e184-87b9-ffff-ffffa7b1fa34')

################### send payload mismatch ######################
b'2\x00\x00E\x100\x120\xe8D]\x03\x009e20cd36e3acc67942c9aa859d3cf193\x00\x03\x00\x00\x00'
b'\x1a\x00\x00E\x100\x120{\x00\x00\x00\x00password\x00\x03\x00\x00\x00'
MAIN_GS_LOGON|SUB_GS_LOGON_USERID ==> {'id': 0, 'userID': 56444136, 'body_len': 50, 'password': ('9e20cd36e3acc67942c9aa859d3cf193', 32), 'pwd': ''}
MAIN_GS_LOGON|SUB_GS_LOGON_USERID ==> {'id': 0, 'userID': 123, 'body_len': 26, 'password': ('password', 8), 'pwd': ''}
----------------------------------------------------------------
==>  b'\x08\x00\x00E\x100\x130'
MAIN_GS_LOGON|SUB_GS_LOGON_SUCCESS ==> {'body_len': 8}
==>  b'\x12\x00\x01EP0Q0\x95CI\x00"\x00\x14\x00\x02\x00'
MAIN_GS_INFO|SUB_GS_SERVER_INFO ==> {'serverID': 17301, 'chairCount': 2, 'kindID': 73, 'tableCount': 20, 'body_len': 18, 'serverType': 34}
==>  b'x\x00\x02E 0%0\x01\x00\x04\x00\x00\x00\x00\x00\x00\x00\xe8D]\x03\xdc\x07\x00\x00\n\x00\xff\xff\xff\xff\x00\x00\x01\x08\x00\x00\x00\x00\x00\x00\x00\xfe

==========> We draw  3条
=======> my hand:  1条|2条|3条|9条|9条|2万|8万|8万|2筒|8筒|8筒|9筒|9筒|混|
[0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
##############=============> We suggest 2万 times[16]     shanten[2]  waiting[8万|7筒|8筒|9筒|9条]  desperate[8万|7筒|8筒|9筒|9条]
################### send payload matched ######################
MAIN_GS_GAME|SUB_C_OUT_CARD ==> {'tile': (18,), 'body_len': 9}
***************************************************************
MAIN_GS_GAME|SUB_S_OUT_CARD ==> {'tile': 18, 'body_len': 11, 'user': 0}
==========> We out  2万
=======> my hand:  1条|2条|3条|9条|9条|8万|8万|2筒|8筒|8筒|9筒|9筒|混|
MAIN_GS_GAME|SUB_S_SEND_CARD ==> {'tile': 0, 'body_len': 29, 'user': 1}
MAIN_GS_GAME|SUB_S_OPERATE_RESULT ==> {'cbOperateCard': 36, 'wProvideUser': 1, 'body_len': 26, 'wOperateUser': 1, 'cbOperateCode': 16}
MAIN_GS_GAME|SUB_S_SEND_CARD ==> {'tile': 0, 'body_len': 29, 'user': 1}
HEAT_BEAT|HEAT_BEAT ==> {'body_len': 16, 'raw': b'(\x80\x16[\x00\x00\

In [11]:
cc = PcapGameConnection(filename='/tmp/0426.pcap')
# rdpcap?

cc = PcapGameConnection(live=1)

cc = None

while True:
    print(decode_payload(cc.get_message()[0]))

UnboundLocalError: local variable 'data' referenced before assignment

In [ ]:
TERMS = set((0,8,9,17,18,26,27,28,29,30,31,32,33))

def terminals(hand):
    c = {}
    for i in hand:
        if i in TERMS:
            if i in c:
                c[i] +=1
            else:
                c[i] = 1
    d = [i for i in c  if c[i] > 1]
    return len(c), len(d)

def has_shun(hand, tile):
    if tile >= 25:
        return False
    if tile%9 > 6:
        return False
    return tile+1 in hand and tile+2 in hand


def proto_group(ll, hand, gg, pp, tt):
    i = 0
    l = len(hand)
    ret = 8
    found = False
    for i in range(l-1):
        if hand[i] == hand[i+1]:
            sub_hand = hand[:i]+hand[i+2:]
            found = True
            ret = min(ret, proto_group(ll,sub_hand,gg,pp+1,tt))
        elif hand[i+1] < 27 and (hand[i+1]-hand[i]) < 3 and hand[i]//9 == hand[i+1]//9:
            sub_hand = hand[:i]+hand[i+2:]
            found = True
            ret = min(ret, proto_group(ll,sub_hand,gg,pp,tt+1))
    if not found:
        if pp+tt > ll//3:
            if pp > 0:
                ret = min(ret, 8 - 2*gg - ll//3 - 1)
            else:
                ret = min(ret, 8 - 2*gg - ll//3 - 0)
        else:
            ret = min(ret, 8 - 2*gg - pp - tt) 
#     if ret == 3:
#         print(gg, pp, tt)
    return ret


def shanten_list(hand, g):
    ret = 8
    t = 0
    p = 0
#     if len(hand) >= 13:
#         diffTerminals, terminalPairs = terminals(hand)
#         ret = min(ret, 13 - diffTerminals -min(1, terminalPairs))
    l = len(hand)
    if l >= 3:
        for i in range(0,l-2):
            if hand[i]*2 == hand[i+1]+hand[i+2]:
                sub_hand = hand[0:i]+hand[i+3:]
            elif has_shun(hand, hand[i]):
                sub_hand = list(hand)
                sub_hand.remove(hand[i])
                sub_hand.remove(hand[i]+1)
                sub_hand.remove(hand[i]+2)
            else:
                continue
            ret = min(ret, shanten_list(sub_hand, g+1))

    if p == 7:
        return -1
    if l >= 13:
        c = [0]*34
        for i in hand:
            c[i] += 1
        ret = min(ret, 6 - len([i for i in c if i > 1]))

    ret = min(ret, proto_group(l, hand, g, p, t))
    
    if p+t > l//3:
#         print(222)
        if p > 0:
            ret = min(ret, 8 - 2*g - l//3 - 1)
        else:
            ret = min(ret, 8 - 2*g - l//3 - 0)
    else:
        ret = min(ret, 8 - 2*g - p - t)
        
        

    return ret

def shanten_3(hand, g):
    h = []
    for i in range(34):
        if hand[i]:
            h += [i]*hand[i]
#     print(h)
    return shanten_list(h, g)

In [75]:
from mahjong.tests_mixin import TestMixin
from mahjong.utils import find_isolated_tile_indices
from mahjong.tile import TilesConverter
import mahjong.utils

a = TestMixin()
# tiles = a._string_to_136_array(sou='11134567', pin='159', man='45')
tiles_34 = a._string_to_34_array(sou='9999', man='234', pin='', honors='')

t = TilesConverter.string_to_136_array(man='3')[0]//4
print(t)
print(tiles_34)
# should_op_tile(tiles_34, [0]*34, t)
# shanten(tiles_34, 1)
calculate_outs(tiles_34, [0]*34)
# # # discard_tile(tiles_34, [0]*34)



# d = [0] * 34

# h = tiles_34

# should_op_tile(h, d, t)

# do = discard_tile(tiles_34, [0]*34)
# # give_me_tile(do['waiting'], tiles_34, d)
# do

2
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]


[]

In [10]:
TilesConverter.to_one_line_string([i*4 for i in [12, 13, 14, 15, 19]])

'4567p2s'

In [9]:
%debug

> <ipython-input-2-cdfabd673eea>(55)should_op_tile()
     53         opt = rs[0]
     54 #         print(opt)
---> 55         if opt['shanten'] < best_opt['shanten'] or opt['tiles_count'] > best_opt['tiles_count']:
     56             best_opt = opt
     57             ret = (tile+1, tile+2)

ipdb> opt
{'waiting': [1], 'tiles_count': 99, 'discard': 2, 'shanten': 0}
ipdb> best_opt
ipdb> best_opt
ipdb> quit


In [108]:
t = TilesConverter.string_to_136_array
struct.pack?

In [2]:
from codes import *

import struct
import traceback

g = None
enemy_id = 0

def encode_message(payload, cmd_main, cmd_sub):
    frame_len = len(payload) + 8
    header = struct.pack('HBbHH', frame_len, 0, 69, int(cmd_main), int(cmd_sub))
    return header+payload

def decode_payload(data, server = True):
#     ws_op, ws_len = struct.unpack('BB', data[:2])
#     ll = 0
#     if ws_len == 126:
# #             print('got it')
#         ll = 2
    global hand_cards, g, enemy_id
    frame_len, check_code, v, cmd_main, cmd_sub = struct.unpack('HBbHH', data[:8])
    payload = data[8:]
    print((frame_len, cmd_main, cmd_sub))
    try:
        print((codes[cmd_main], codes[cmd_sub]))
        if codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_START_SEND_CARD':
            cards = decode_MAIN_GS_GAME_SUB_S_START_SEND_CARD(payload)
            print(cards)
            hand_cards = cards
            g = Game()
            if cards[13] == 0: 
#                 print('66666666666666666666666666666666666666')
                del cards[13]
                g.me_init_hand(cards)
            else: ## we should out card
                g.me_init_hand(cards)
                g.me_worst_tile()
#                 out_card()
        elif codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_OUT_CARD':
            user, card = decode_MAIN_GS_GAME_SUB_S_OUT_CARD(payload)
            print(user, card, CARDS[card])
            print(CARDS[card])
            if user == enemy_id:
                g.enemy_out(card)
            else:
                g.me_out(card)
        elif codes[cmd_main] == 'MAIN_GS_USER' and codes[cmd_sub] == 'SUB_GS_USER_SCORE':
            info = decode_MAIN_GS_USER__SUB_GS_USER_SCORE(payload)
            print(info)
        elif codes[cmd_main] == 'MAIN_GS_USER' and codes[cmd_sub] == 'SUB_GS_USER_COME':
            info = decode_MAIN_GS_USER__SUB_GS_USER_COME(payload)
            enemy_id = info['wChairID']
            if enemy_id != 0: enemy_id =1
            print(info)
        elif codes[cmd_main] == 'MAIN_GS_USER' and codes[cmd_sub] == 'SUB_GS_USER_STATUS':
            info = SUB_GS_USER_STATUS()
            info.decode_body(payload)
            print(info)
        elif codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_SEND_CARD':
            user, card = decode_MAIN_GS_GAME_SUB_S_SEND_CARD(payload)
            print(user, card, CARDS[card])
            print(CARDS[card])
            if user != enemy_id:
                g.me_draw_tile(card)
                g.me_worst_tile()
#                 out_card()
        elif codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_TRUSTEE':
            bTrustee, wChairID = decode_MAIN_GS_GAME_SUB_S_TRUSTEE(payload)
            a = 'we' if wChairID == enemy_id else 'they'
            b = '托管' if bTrustee else '取消托管'
            print('======> %s %s' % (a, b))
        elif codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_OPERATE_NOTIFY':
            wResumeUser, wProvideUser, cbActionMask, cbActionCard, bIsQiangGangHu = decode_MAIN_GS_GAME_SUB_S_OPERATE_NOTIFY(payload)
            print(wResumeUser, wProvideUser, cbActionMask, cbActionCard, bIsQiangGangHu)
            m = cbActionMask
            actions = WIK_LEFT | WIK_CENTER | WIK_RIGHT | WIK_PENG
            action = 0;
#             print('55555555555555555555555')
            print(actions, m, actions&m)
            if actions & m:
                action = g.operate_tile(cbActionCard, m)
            if m & WIK_GANG:
                g.no_op()
            if m & WIK_CHI_HU:
                g.chi_hu()
            if m & WIK_ZI_MO:
                g.zi_mo()
#             if action:
        elif codes[cmd_main] == 'MAIN_GS_GAME' and codes[cmd_sub] == 'SUB_S_OPERATE_RESULT':
            wOperateUser, wProvideUser, cbOperateCode, cbOperateCard = decode_MAIN_GS_GAME_SUB_S_OPERATE_RESULT(payload)
            print(wOperateUser, wProvideUser, cbOperateCode, cbOperateCard)
            cards = []
            if cbOperateCode == WIK_LEFT:
                cards = [cbOperateCard+1, cbOperateCard+2]
            elif cbOperateCode == WIK_CENTER:
                cards = [cbOperateCard-1, cbOperateCard+1] 
            elif cbOperateCode == WIK_RIGHT:
                cards = [cbOperateCard-2, cbOperateCard-1] 
            elif cbOperateCode == WIK_PENG:
                cards = [cbOperateCard, cbOperateCard] 
            elif cbOperateCode == WIK_GANG:
                cards = [cbOperateCard, cbOperateCard, cbOperateCard]
            if wOperateUser == enemy_id:
                for i in cards:
                    g.enemy_out(i)
            else:
                for i in cards:
                    g.me_out(i)
                if cbOperateCode: g.me_worst_tile()
#             m = cbActionMask
#             actions = WIK_LEFT | WIK_CENTER | WIK_RIGHT | WIK_PENG
#             action = 0;
#             print('55555555555555555555555')
#             print(actions, m, actions&m)
#             if actions & m:
#                 action = g.operate_tile(cbActionCard, m)
        elif codes[cmd_main] == 'MAIN_GS_LOGON' and codes[cmd_sub] == 'SUB_GS_LOGON_USERID':
            print(decode_MAIN_GS_LOGON__SUB_GS_LOGON_USERID(payload))
    except:
        print('###################################################################################')
        print('###################################################################################')
        traceback.print_exc()
        print('oops')
#     data = data[frame_len+2+ll:]

def decode_MAIN_GS_GAME_SUB_S_START_SEND_CARD(data):
#     print(data)
    cards = []
    for i in data[4:18]:
        if type(i) == int:
            cards.append(i)
        else:
            cards.append(ord(i))
    return cards

def read_str(buff):
    n = buff.find(b'\x00')
    return buff[0:n].decode('gbk'), n

def decode_MAIN_GS_GAME_SUB_S_OUT_CARD(data):
#     print(data)
    user, card = struct.unpack('HB', data[:3])
    return user, card

def decode_MAIN_GS_LOGON__SUB_GS_LOGON_USERID(data):
#     print(data)
    userID = struct.unpack('I', data[:4])[0]
    password = read_str(data[5:])
    return userID, password
def encode_MAIN_GS_LOGON__SUB_GS_LOGON_USERID(userID, password):
#     print(data)
    ret = b''
    ret += struct.pack('I', userID)
    ret += b'\x00'
    ret += bytes(password, 'utf-8')
    ret += b'\x00'
    ret += struct.pack('I', 3)
    return ret

def decode_MAIN_GS_GAME_SUB_S_SEND_CARD(data):
#     print(data)
    user = struct.unpack('H', data[18:20])[0]
    card = struct.unpack('B', data[:1])[0]
    return user, card

def decode_MAIN_GS_GAME_SUB_S_TRUSTEE(data):
#     print(data)
    bTrustee, wChairID = struct.unpack('<bH', data)
    return bTrustee, wChairID

def decode_MAIN_GS_USER__SUB_GS_USER_SCORE(data):
    k = ('dwUserID', 'lGold', 'lScore', 'lWinCount', 'lLostCount', 'lDrawCount', 
          'lFleeCount', 'lExperience', 'lCharm', 'lGameCount', 'lGameCount_All', 'lWorkCount')
    return dict(zip(k,struct.unpack('iiiiiiiiiiii',data)))

def decode_MAIN_GS_USER__SUB_GS_USER_COME(data):
    k = ('wFaceID', 'cbGender', 'cbMember', 'lCharm', 'id', 'lBirthday_year', 'lBirthday_month', 
         'lBirthday_day', 'wTableID', 'wChairID', 'wNetDelay', 'cbUserStatus', 'dwUserRight', 
         'dwMasterRight', 'lGold', 'lScore', 'lWinCount', 'lLostCount', 'lDrawCount', 'lFleeCount', 
         'lExperience', 'onlineTime', 'lGameCount_Work', 'lLotteryCnt', 'lGameCount', 'lGameCount_All ', 'lNewMailCount')
    info = dict(zip(k,struct.unpack('<HbBIIIbbHHHbIIIIIIIIIIIIIII',data[2:2+85])))
    cursor = 2+85
    s, n = read_str(data[cursor:])
    info['szAccount'] = s
    cursor = cursor + n + 1
    s, n = read_str(data[cursor:])
    info['nickName'] = s
    cursor = cursor + n + 1
    s, n = read_str(data[cursor:])
    info['szBlog'] = s
    cursor = cursor + n + 1
    s, n = read_str(data[cursor:])
    info['Region0'] = s
    cursor = cursor + n + 1
    s, n = read_str(data[cursor:])
    info['Region1'] = s
    cursor = cursor + n + 1
    return info

def decode_MAIN_GS_GAME_SUB_S_OPERATE_NOTIFY(data):
#     print(data)
    wResumeUser, wProvideUser, cbActionMask, cbActionCard, bIsQiangGangHu = struct.unpack('HHBBB', data[:7])
    return wResumeUser, wProvideUser, cbActionMask, cbActionCard, bIsQiangGangHu

def decode_MAIN_GS_GAME_SUB_S_OPERATE_RESULT(data):
#     print(data)
    wOperateUser, wProvideUser, cbOperateCode, cbOperateCard = struct.unpack('hhbb', data[:6])
    return wOperateUser, wProvideUser, cbOperateCode, cbOperateCard

##########################################################################################
def unmask_payload(data, masks):
    ret = b''
    for i in range(len(data)):
        ret += struct.pack('B', ord(data[i:i+1])^ord(masks[i%4:i%4+1]))
    return ret

from scapy.all import *
class TcpStream():
    def __init__(self, pkt):
        self.init = True
        if 'Raw' in pkt:
#             self.data = pkt[Raw].load
            self.data = b''
            self.nseq = pkt[TCP].seq + len(self.data)
        else:
            self.data = b''
            if pkt[TCP].flags & 2 == 2:
                self.nseq = pkt[TCP].seq  + 1
    def add_packet(self, pkt):
        if pkt[TCP].seq != self.nseq:
            print(pkt[TCP].seq, self.nseq)
            print('Oops')
            return
        if 'Raw' in pkt:
            self.data += pkt[Raw].load
            self.nseq = pkt[TCP].seq + len(pkt[Raw].load)
            if self.init:
                head = self.data.find(b'\r\n\r\n')
                if head > 0:
                    begin = head+4
                    self.init = False
                    self.c = begin
            if not self.init:
#                 print('**********************************************')
                for i in range(5):
                    found_frame = False
#                     print (len(self.data) - self.c - 1, len(self.data), self.c)
                    if len(self.data) - self.c >= 2:
#                         print('1111111111111111111111111111111111')
                        ws_op, ws_len = struct.unpack('BB', self.data[self.c:self.c+2])
                        ll = 0
                        mm = 0
                        mask_flag = ws_len & 0x80
                        ws_len = ws_len & 0x7f
                        if mask_flag > 0: mm = 4
                        if ws_len == 126:
    #                         print('*******', self.data[self.c:self.c+6])
                            ll = 2
                            if len(self.data) - self.c >= 2+ll+4:
                                ws_len = struct.unpack('>H', self.data[self.c+2:self.c+4])[0]
#                         print(self.data)
#                         print(self.c, ll, mm, ws_len)
                        if len(self.data) - self.c >= 2+ll+mm+ws_len:
#                             print('22222222222222222222222222222222')
#                             print('GOT WS Frame ==>', mm, self.data[self.c:self.c+2+ll+mm+ws_len])
                            payload = self.data[self.c+2+ll+mm:self.c+2+ll+mm+ws_len]
                            if mask_flag == 0:
                                masks = b'\x00\x00\x00\x00'
                            else:
                                masks = self.data[self.c+2+ll:self.c+2+ll+4]
                            if mask_flag: 
                                payload = unmask_payload(payload, masks)
                                print('MASKED DATA ======>', payload)
                            if ws_op == 0x82:
                                decode_payload(payload, server = (mask_flag == 0))
                            found_frame = True
                            self.c += 2+ll+mm+ws_len
                            self.data = self.data[self.c:]
                            self.c = 0
                    if not found_frame:
#                         print('##################')
                        break
#                     else:
#                         print('##################')
#                         break
#             print('====>', pkt[TCP].seq, len(pkt[Raw].load))
        else:
#             self.data = ''
            if pkt[TCP].flags & 3 : # SYN or FIN
                self.nseq = pkt[TCP].seq  + 1

streams = {}
def deal_pkt(p):
    if (p[TCP].sport, p[TCP].dport) not in streams:
        s = TcpStream(p)
        streams[(p[TCP].sport, p[TCP].dport)] = s
    else:
        s = streams[(p[TCP].sport, p[TCP].dport)]
        s.add_packet(p)

import socket

def get_pkt():
    global streams
    streams = {}
    sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sk.bind(('0.0.0.0', 8989))
    try:
        while True:
            data, addr = sk.recvfrom(8192)
            p = Ether(data)
            deal_pkt(p)
    except:
        traceback.print_exc()
        sk.close()
        sk = None

from termcolor import colored, cprint
class Game():
    def __init__(self):
        self.hand = []
        self.me_out_list = []
        self.enemy_out_list = []
        self.double = 0
    def me_out(self, tile):
        self.me_out_list.append(tile)
        self.hand.remove(tile)
        print('==========> We out ', CARDS[tile])
        self.print_myhand()
    def me_worst_tile(self):
        t = None
        if 0x35 in self.hand:
            t = 0x35
        if 0x36 in self.hand:
            t = 0x36
        if 0x37 in self.hand:
            t = 0x37
        if not t:
            self.print_myhand()
            print(list_to_34(self.hand))
            t = discard_tile(list_to_34(self.hand), list_to_34(self.me_out_list+self.enemy_out_list))
            t = tile_from_34[t]
        cprint('##############=============> We suggest '+ CARDS[t], 'green')
        return t
    def enemy_out(self, tile):
        self.enemy_out_list.append(tile)
        print('==========> They out ', CARDS[tile])
    def me_init_hand(self, tiles):
        self.hand = tiles
    def me_draw_tile(self, tile):
        self.hand.append(tile)
        print('==========> We draw ', CARDS[tile])
    def operate_tile(self, tile, mask):
        opt = should_op_tile(list_to_34(self.hand), list_to_34(self.me_out_list+self.enemy_out_list), tile_to_34[tile])
        if opt:
            t1, t2 = [tile_from_34[i] for i in opt]
            rs = [t1, t2, tile]
            rs.sort()
            rs = [CARDS[i] for i in rs]
            cprint('##############=============> We suggest '+ ''.join(rs), 'green')
        else:
            cprint('##############=============> We suggest 过', 'green')
    def print_myhand(self):
        self.hand.sort()
        ret = ''
        for i in self.hand:
            ret += CARDS[i]+'|'
        print('=======> my hand: ', ret)
    def no_op(self):
        cprint('##############=============> We suggest 过', 'green')
    def chi_hu(self):
        cprint('##############=============> We suggest 胡', 'green')
    def zi_mo(self):
        cprint('##############=============> We suggest 自摸', 'green')


In [167]:
c = b'7\x00\x00E\x100\x140\x01\x00\x00\x00\xd5\xca\xba\xc5\xbb\xf2\xd5\xdf\xc3\xdc\xc2\xeb\xd3\xd0\xce\xf3\xa3\xac\xc7\xeb\xb3\xa2\xca\xd4\xd6\xd8\xd0\xc2\xb5\xc7\xc2\xbc\xd3\xce\xcf\xb7\xb4\xf3\xcc\xfc\xa3\xa1\x00'
# decode_payload(c)
PKT.decode(c)

MAIN_GS_LOGON|SUB_GS_LOGON_FAIL ==> {'body_len': 55, 'errorCode': 1, 'errorDesc': '帐号或者密码有误，请尝试重新登录游戏大厅！'}

In [180]:
# e = b'2\x00\x00E\x100\x120_\xcf_\x03\x00034225f4edd7a6ce9814e8b63406efa2\x00\x03\x00\x00\x00'
# p = encode_MAIN_GS_LOGON__SUB_GS_LOGON_USERID(56610655, '034225f4edd7a6ce9814e8b63406efa2')
# m = encode_message(p, codes_c2n['MAIN_GS_LOGON'], codes_c2n['SUB_GS_LOGON_USERID'])
# assert m == e

l_p = SUB_GS_LOGON_USERID(56610655, '034225f4edd7a6ce9814e8b63406efa2')
dir(l_p)
# print(l_p)

play_game('hello')

################### send payload mismatch ######################
b'2\x00\x00E\x100\x120\xe8D]\x03\x009e20cd36e3acc67942c9aa859d3cf193\x00\x03\x00\x00\x00'
b'\x1a\x00\x00E\x100\x120{\x00\x00\x00\x00password\x00\x03\x00\x00\x00'
MAIN_GS_LOGON|SUB_GS_LOGON_USERID ==> {'body_len': 50, 'pwd': '', 'id': 0, 'userID': 56444136, 'password': ('9e20cd36e3acc67942c9aa859d3cf193', 32)}
MAIN_GS_LOGON|SUB_GS_LOGON_USERID ==> {'body_len': 26, 'pwd': '', 'id': 0, 'userID': 123, 'password': ('password', 8)}
----------------------------------------------------------------
==>  b'\x08\x00\x00E\x100\x130'
MAIN_GS_LOGON|SUB_GS_LOGON_SUCCESS ==> {'body_len': 8}
==>  b'\x12\x00\x01EP0Q0\x95CI\x00"\x00\x14\x00\x02\x00'
MAIN_GS_INFO|SUB_GS_SERVER_INFO ==> {'body_len': 18, 'kindID': 73, 'serverType': 34, 'tableCount': 20, 'serverID': 17301, 'chairCount': 2}
==>  b'x\x00\x02E 0%0\x01\x00\x04\x00\x00\x00\x00\x00\x00\x00\xe8D]\x03\xdc\x07\x00\x00\n\x00\xff\xff\xff\xff\x00\x00\x01\x08\x00\x00\x00\x00\x00\x00\x00\xfe

In [10]:
%debug

> <ipython-input-1-d3fcedb2f631>(153)body()
    151         self.action, self.tile = struct.unpack('BB', data)
    152     def body(self):
--> 153         return struct.pack('BB', self.action, self.tile)
    154 PKT.register(SUB_C_OPERATE_CARD)
    155 

ipdb> self
MAIN_GS_GAME|SUB_C_OPERATE_CARD ==> {'action': None, 'tile': 1}
ipdb> quit


In [10]:
login_3('00000000-8eba-2b33-ffff-ffff78dba179')

{'code': 1, 'msg': '您的IP已禁止注册'}

In [6]:
%debug

> <ipython-input-3-d17483a69091>(12)login()
     10     content = conn.getresponse().read()
     11     content = json.loads(content.decode('utf-8'))
---> 12     headers = {"Cookie":"JSESSIONID=%s"%content['sessionid'],"session_id":"%s"%content['sessionid'],
     13               "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.1.1; SM-N9005 Build/NJH47F)"}
     14     if content['score'] < 500:

ipdb> content
{'msg': '您的帐号已冻结，请联系客服', 'code': 1}
ipdb> quit


In [25]:
# import importlib
# importlib.reload('websocket')
from websocket import create_connection
ws = create_connection("ws://73.hanyou.com:17311/")
ws.send(m)
result =  ws.recv()
print(result)
decode_payload(result)
ws.close()

b')\x00\x00E\x800\x810\x03P\xc4\xfa\xd6\xc1\xc9\xd9\xd0\xe8\xd2\xaa 50000 \xd2\xf8\xc1\xbd\xb2\xc5\xc4\xdc\xbd\xf8\xc8\xeb!\x00'
(41, 12416, 12417)
('SUB_GS_GAME_DADISHU_INFO', 'SUB_GS_GAME_DADISHU_HIT_REQ')


In [13]:
print(result[8:])
a = b'\xd5\xca\xba\xc5\xbb\xf2\xd5\xdf\xc3\xdc\xc2\xeb\xd3\xd0\xce\xf3\xa3\xac\xc7\xeb\xb3\xa2\xca\xd4\xd6\xd8\xd0\xc2\xb5\xc7\xc2\xbc\xd3\xce\xcf\xb7\xb4\xf3\xcc\xfc\xa3\xa1'
a.decode('gbk')

b'\x01\x00\x00\x00\xd5\xca\xba\xc5\xbb\xf2\xd5\xdf\xc3\xdc\xc2\xeb\xd3\xd0\xce\xf3\xa3\xac\xc7\xeb\xb3\xa2\xca\xd4\xd6\xd8\xd0\xc2\xb5\xc7\xc2\xbc\xd3\xce\xcf\xb7\xb4\xf3\xcc\xfc\xa3\xa1\x00'


'帐号或者密码有误，请尝试重新登录游戏大厅！'

In [92]:
streams = {}
sniff(prn=deal_pkt, offline='/tmp/0605.pcap')
# get_pkt()

MASKED DATA ======> b'2\x00\x00E\x100\x120\xe8D]\x03\x009e20cd36e3acc67942c9aa859d3cf193\x00\x03\x00\x00\x00'
(50, 12304, 12306)
('MAIN_GS_LOGON', 'SUB_GS_LOGON_USERID')
(56444136, ('9e20cd36e3acc67942c9aa859d3cf193', 32))
(8, 12304, 12307)
('MAIN_GS_LOGON', 'SUB_GS_LOGON_SUCCESS')
(18, 12368, 12369)
('MAIN_GS_INFO', 'SUB_GS_SERVER_INFO')
(120, 12320, 12325)
('MAIN_GS_USER', 'SUB_GS_USER_COME')
{'lBirthday_month': 10, 'wTableID': 65535, 'lGameCount_Work': 0, 'Region1': '上海', 'wChairID': 65535, 'dwMasterRight': 0, 'szBlog': '', 'lGameCount_All ': 0, 'cbMember': 0, 'wFaceID': 4, 'lGold': 24062, 'cbUserStatus': 1, 'lNewMailCount': 0, 'lLotteryCnt': 0, 'id': 56444136, 'lBirthday_day': 0, 'lBirthday_year': 2012, 'cbGender': 0, 'dwUserRight': 8, 'lExperience': 0, 'onlineTime': 0, 'lGameCount': 0, 'wNetDelay': 0, 'lScore': 0, 'Region0': '上海', 'lWinCount': 2415, 'szAccount': '', 'lCharm': 0, 'lDrawCount': 1, 'lLostCount': 1934, 'nickName': '上海56444136', 'lFleeCount': 0}
(8, 12304, 12309)
('MAI

=======> my hand:  1条|2条|3条|5条|9条|9条|8万|8万|8筒|8筒|9筒|9筒|混|
(29, 12448, 102)
('MAIN_GS_GAME', 'SUB_S_SEND_CARD')
1 0 未知
未知
(11, 12448, 101)
('MAIN_GS_GAME', 'SUB_S_OUT_CARD')
1 1 1条
1条
==========> They out  1条
(15, 12448, 104)
('MAIN_GS_GAME', 'SUB_S_OPERATE_NOTIFY')
0 1 1 1 0
15 1 1
##############=============> We suggest 过
2457170981 2457171011
Oops
MASKED DATA ======> b'\n\x00\x00E\xa00\x03\x00\x00\x00'
(10, 12448, 3)
('MAIN_GS_GAME', 'SUB_C_OPERATE_CARD')
(26, 12448, 105)
('MAIN_GS_GAME', 'SUB_S_OPERATE_RESULT')
0 0 0 0
(29, 12448, 102)
('MAIN_GS_GAME', 'SUB_S_SEND_CARD')
0 6 6条
6条
==========> We draw  6条
=======> my hand:  1条|2条|3条|5条|6条|9条|9条|8万|8万|8筒|8筒|9筒|9筒|混|
[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
##############=============> We suggest 6条
(16, 0, 1)
('GUESS_HEART_BEAT', 'SUB_C_OUT_CARD')
MASKED DATA ======> b'\x10\x00\x00E\x00\x00\x01\x00G\x80\x16[\x00\x00\x00\x00'
(16, 0, 1)
('GUESS_HEART_BEAT', 'SUB_C_OUT_CARD')

=======> my hand:  1条|4条|6条|6条|1万|3万|5万|2筒|3筒|4筒|7筒|9筒|西|混|
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
##############=============> We suggest 西
MASKED DATA ======> b'\t\x00\x00E\xa00\x01\x003'
(9, 12448, 1)
('MAIN_GS_GAME', 'SUB_C_OUT_CARD')
(11, 12448, 101)
('MAIN_GS_GAME', 'SUB_S_OUT_CARD')
0 51 西
西
==========> We out  西
=======> my hand:  1条|4条|6条|6条|1万|3万|5万|2筒|3筒|4筒|7筒|9筒|混|
(29, 12448, 102)
('MAIN_GS_GAME', 'SUB_S_SEND_CARD')
1 0 未知
未知
(11, 12448, 101)
('MAIN_GS_GAME', 'SUB_S_OUT_CARD')
1 2 2条
2条
==========> They out  2条
(29, 12448, 102)
('MAIN_GS_GAME', 'SUB_S_SEND_CARD')
0 54 翻倍
翻倍
==========> We draw  翻倍
##############=============> We suggest 翻倍
MASKED DATA ======> b'\t\x00\x00E\xa00\x01\x006'
(9, 12448, 1)
('MAIN_GS_GAME', 'SUB_C_OUT_CARD')
(11, 12448, 101)
('MAIN_GS_GAME', 'SUB_S_OUT_CARD')
0 54 翻倍
翻倍
==========> We out  翻倍
=======> my hand:  1条|4条|6条|6条|1万|3万|5万|2筒|3筒|4筒|7筒|9筒|混|
(29, 12448, 102)
('MAIN_GS_GAME',

<Sniffed: TCP:514 UDP:0 ICMP:0 Other:0>

In [20]:
cprint('hello', color='yellow', attrs=['underline'])

hello


In [15]:
cprint

<function termcolor.cprint>